In [1]:
import json
import sys
import time
import warnings
from pathlib import Path
from typing import Literal, Optional

import lightning as L
import torch
from lightning.fabric.strategies import FSDPStrategy

from scripts.prepare_alpaca import generate_prompt
from generate.base import generate
from lit_gpt import GPT, Tokenizer, Config
from lit_gpt.model import Block
from lit_gpt.utils import lazy_load, check_valid_checkpoint_dir, quantization

/home/ec2-user/SageMaker/test/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ec2-user/SageMaker/test/venv/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
torch.set_float32_matmul_precision("high")
input: str = "What is AIPI?"
finetuned_path: Path = Path("./s3/out/full_finetune/lit_model_finetuned.pth")
checkpoint_dir: Path = Path("./s3/checkpoint/meta-llama/Llama-2-7b-hf")
quantize: Optional[Literal["bnb.nf4", "bnb.nf4-dq", "bnb.fp4", "bnb.fp4-dq", "bnb.int8", "gptq.int4"]] = None
strategy: str = "auto"
devices: int = 1
precision: str = "bf16-true"
max_new_tokens: int = 100
temperature: float = 0.8
top_k: Optional[int] = 200

In [3]:
if strategy == "fsdp":
    strategy = FSDPStrategy(auto_wrap_policy={Block}, cpu_offload=False)
fabric = L.Fabric(devices=[7], precision=precision, strategy=strategy)
fabric.launch()

check_valid_checkpoint_dir(checkpoint_dir)

with open(checkpoint_dir / "lit_config.json") as fp:
    config = Config(**json.load(fp))

if quantize is not None:
    # TODO: we need to clean-up the logic for quantizing the finetuned models and loading them after
    raise NotImplementedError
checkpoint_path = finetuned_path

fabric.print(f"Loading model {str(checkpoint_path)!r} with {config.__dict__}", file=sys.stderr)
t0 = time.time()
with fabric.init_module(empty_init=True), quantization(quantize):
    model = GPT(config)
fabric.print(f"Time to instantiate model: {time.time() - t0:.02f} seconds.", file=sys.stderr)

t0 = time.time()
with lazy_load(finetuned_path) as checkpoint:
    model.load_state_dict(checkpoint.get("model", checkpoint), strict=quantize is None)
fabric.print(f"Time to load the model weights: {time.time() - t0:.02f} seconds.", file=sys.stderr)

model.eval()
model = fabric.setup(model)

tokenizer = Tokenizer(checkpoint_dir)
sample = {"input": input}
prompt = generate_prompt(sample)
encoded = tokenizer.encode(prompt, device=model.device)
prompt_length = encoded.size(0)
max_returned_tokens = prompt_length + max_new_tokens


Loading model 's3/out/full_finetune/lit_model_finetuned.pth' with {'org': 'meta-llama', 'name': 'Llama-2-7b-hf', 'block_size': 4096, 'vocab_size': 32000, 'padding_multiple': 64, 'padded_vocab_size': 32000, 'n_layer': 32, 'n_head': 32, 'n_embd': 4096, 'rotary_percentage': 1.0, 'parallel_residual': False, 'bias': False, 'n_query_groups': 32, 'shared_attention_norm': False, '_norm_class': 'RMSNorm', 'norm_eps': 1e-05, '_mlp_class': 'LLaMAMLP', 'intermediate_size': 11008, 'condense_ratio': 1}
Time to instantiate model: 0.08 seconds.
Time to load the model weights: 9.68 seconds.


In [4]:
prompt

'### Input:\nWhat is AIPI?\n\n### Response:'

In [5]:
y = generate(
    model,
    encoded,
    max_returned_tokens,
    max_seq_length=max_returned_tokens,
    temperature=temperature,
    top_k=top_k,
    eos_id=tokenizer.eos_id,
)

model.reset_cache()
output = tokenizer.decode(y)
output = output.split("### Response:")[1].strip()
fabric.print(output)

ThisA is principleAC of toaus to a starting years and work.
